In [ ]:
from flask import Flask, render_template, jsonify, request
from langchain_pinecone import PineconeVectorStore
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from googleapiclient.discovery import build
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.memory import ConversationBufferWindowMemory
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from paystackapi.paystack import Paystack 
import os

c:\Users\ADEGOKE\anaconda3\envs\Carebot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ADEGOKE\anaconda3\envs\Carebot\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
%pwd

'c:\\Users\\ADEGOKE\\Desktop\\DishDash-AI\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ADEGOKE\\Desktop\\DishDash-AI'

In [5]:
from datasets import load_dataset
import pandas as pd

  # Load the dataset

In [6]:
try:
    dataset_foods = load_dataset("Tinny-Robot/Nigerian-Foods")
    df_foods = pd.DataFrame(dataset_foods['train'])
    
    print("Successfully loaded Tinny-Robot/Nigerian-Foods")
    print(df_foods.head())
except Exception as e:
    print(f"Error loading Tinny-Robot/Nigerian-Foods: {e}")


Successfully loaded Tinny-Robot/Nigerian-Foods
   Id REFID      Code                    Category  \
0   1     1  01_01_01  Cereals and Grain Products   
1   2     3  01_01_03  Cereals and Grain Products   
2   3     2  01_01_02  Cereals and Grain Products   
3   4     4  01_02_01  Cereals and Grain Products   
4   5     5  01_02_02  Cereals and Grain Products   

                                       LocalName  \
0                                           None   
1                                           None   
2                              Tuwon Acha(Hausa)   
3  Masara(Hausa), Agbado(Yoruba), Abakpa(Ibibio)   
4                                  Agidi(Yoruba)   

                                    EnglishName    ScientificName FrenchNames  \
0                 Acha, black, whole grain, Raw  Digitaria iburua        None   
1                 Acha, white, whole grain, Raw  Digitaria exilis        None   
2              Acha, white, whole grain, Boiled  Digitaria iburua        None 

In [7]:
df_foods.head()

,Id,REFID,Code,Category,LocalName,EnglishName,ScientificName,FrenchNames,Source,ENERC_kJ,...,VITDEQ_mcg,VITE_mg,THIA_mg,RIBF_mg,NIAEQ_mg,VIT_B6_mg,FOL_mcg,VITB12_mcg,VITC_mg,SearchName
0,1,1,01_01_01,Cereals and Grain Products,None,"Acha, black, whole grain, Raw",Digitaria iburua,None,002,1491.700,...,NaN,0.05,0.260,0.100,1.700,0.740,29.0,NaN,NaN,"Acha, black, whole grain, Raw Digitaria iburua"
1,2,3,01_01_03,Cereals and Grain Products,None,"Acha, white, whole grain, Raw",Digitaria exilis,None,002,1471.000,...,NaN,0.05,0.250,0.100,1.700,0.730,29.0,NaN,NaN,"Acha, white, whole grain, Raw Digitaria exilis"
2,3,2,01_01_02,Cereals and Grain Products,Tuwon Acha(Hausa),"Acha, white, whole grain, Boiled",Digitaria iburua,None,002,613.500,...,NaN,0.02,0.070,0.030,0.600,0.190,9.0,NaN,NaN,"Acha, white, whole grain, Boiled Tuwon Acha(H..."
3,4,4,01_02_01,Cereals and Grain Products,"Masara(Hausa), Agbado(Yoruba), Abakpa(Ibibio)","Maize, white, whole kernel, dried, raw",Zea mays,None,"001,002,007,040,189,203,085,100",1584.231,...,NaN,1.30,0.382,0.156,2.814,0.622,26.0,NaN,5.7,"Maize, white, whole kernel, dried, raw Masara..."
4,5,5,01_02_02,Cereals and Grain Products,Agidi(Yoruba),"Maize, white,fermented gruel,cooked, moulded",None,None,"096,033,035,002,085",395.437,...,NaN,0.04,0.020,0.050,0.300,0.020,1.1,NaN,2.0,"Maize, white,fermented gruel,cooked, moulded ..."


In [8]:
df_foods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              281 non-null    int64  
 1   REFID           281 non-null    object 
 2   Code            281 non-null    object 
 3   Category        281 non-null    object 
 4   LocalName       148 non-null    object 
 5   EnglishName     281 non-null    object 
 6   ScientificName  142 non-null    object 
 7   FrenchNames     0 non-null      object 
 8   Source          280 non-null    object 
 9   ENERC_kJ        281 non-null    float64
 10  ENERC_kcal      281 non-null    float64
 11  WATER_g         278 non-null    float64
 12  PROTCNT_g       274 non-null    float64
 13  FATCE_g         273 non-null    float64
 14  CHOCDF_g        251 non-null    float64
 15  FIB_g           200 non-null    float64
 16  ASH_g           269 non-null    float64
 17  Ca_mg           254 non-null    flo

In [9]:
df_foods.isnull().sum()

Id                  0
REFID               0
Code                0
Category            0
LocalName         133
EnglishName         0
ScientificName    139
FrenchNames       281
Source              1
ENERC_kJ            0
ENERC_kcal          0
WATER_g             3
PROTCNT_g           7
FATCE_g             8
CHOCDF_g           30
FIB_g              81
ASH_g              12
Ca_mg              27
Fe_mg              30
Mg_mg              63
P_mg               47
K_mg               41
Na_mg              54
Zn_mg              64
Cu_mg              88
Mn_mg             158
VIT_A_RAE_mcg     182
RETOL_mcg         247
CARTB_mcg         170
VITDEQ_mcg        252
VITE_mg           164
THIA_mg           108
RIBF_mg            99
NIAEQ_mg          126
VIT_B6_mg         135
FOL_mcg           139
VITB12_mcg        229
VITC_mg           148
SearchName          0
dtype: int64

In [10]:
df_foods.describe()

,Id,ENERC_kJ,ENERC_kcal,WATER_g,PROTCNT_g,FATCE_g,CHOCDF_g,FIB_g,ASH_g,Ca_mg,...,CARTB_mcg,VITDEQ_mcg,VITE_mg,THIA_mg,RIBF_mg,NIAEQ_mg,VIT_B6_mg,FOL_mcg,VITB12_mcg,VITC_mg
count,281.000000,281.000000,281.000000,278.000000,274.000000,273.000000,251.000000,200.000000,269.000000,254.000000,...,111.000000,29.000000,117.000000,173.000000,182.000000,155.000000,146.000000,142.000000,52.000000,133.000000
mean,141.000000,864.247915,205.986082,55.626691,10.608942,8.264110,24.584283,3.250925,3.468665,107.522472,...,1488.868126,2.450345,1.025624,9.267312,6.544599,2.402877,0.318295,57.898923,168.174788,27.570940
std,81.261922,720.915178,173.373893,31.152408,11.483173,15.954411,24.381290,3.762010,19.605722,309.451796,...,6715.906143,3.159067,2.425481,119.121331,85.153355,3.571652,0.337935,108.134528,820.345962,49.912205
min,1.000000,7.480000,1.760000,0.020000,0.100000,0.063000,0.130000,0.020000,0.010000,0.004000,...,0.005000,0.100000,0.004000,0.004000,0.001000,0.024000,0.005000,0.011000,0.020000,0.200000
25%,71.000000,347.900000,82.300000,18.800000,2.025000,0.498000,5.729000,1.000000,0.900000,10.000000,...,24.870000,0.440000,0.150000,0.048000,0.040000,0.500000,0.101000,8.000000,0.575000,3.700000
50%,141.000000,571.900000,135.500000,68.900000,6.767500,2.060000,15.740000,2.010000,1.500000,26.000000,...,126.000000,1.200000,0.330000,0.094000,0.092500,1.000000,0.219000,22.000000,2.025000,11.100000
75%,211.000000,1406.207000,333.190000,79.222500,16.957500,7.130000,32.620000,4.025000,2.580000,82.000000,...,611.150000,3.000000,1.040000,0.220000,0.230000,2.982500,0.383000,52.750000,8.662500,34.333000
max,281.000000,3700.000000,900.000000,99.090000,69.710000,100.000000,99.980000,28.600000,319.160000,3760.000000,...,68680.000000,12.500000,17.200000,1567.000000,1149.000000,28.173000,2.270000,677.000000,4230.000000,326.000000


In [11]:
try:
    dataset_dishes = load_dataset("Nnobody/Nigerian-Dishes")
    df_dishes= pd.DataFrame(dataset_dishes["train"])
    print("\nSuccessfully loaded Nnobody/Nigerian-Dishes")
except Exception as e:
    print(f"Error loading Nnobody/Nigerian-Dishes: {e}")


Successfully loaded Nnobody/Nigerian-Dishes


In [12]:
df_dishes.head()

,Food_Name,Main_Ingredients,Description,Food_Health,Food_Class,Region,Spice_Level,Price_Range
0,Abacha,"African salad, utazi leaves, oil, fish",Cassava-based salad with spicy fish,Healthy,Traditional,South-East,Spicy,Affordable
1,Abacha and Ugba (Regular),"Abacha (Cassava Fufu), Ugba (Oilbean Seeds)",Shredded cassava fufu served with oilbean seeds.,Healthy,Traditional,Eastern,None,Affordable
2,Abacha and Ugba,"African salad, oil, fish, fermented oil bean s...",Cassava-based salad with spicy fish and fermen...,Healthy,Salad,South-East,Spicy,Affordable
3,Afang Soup (Spicy),"Afang Leaves, Water yam, Seafood","Spicy soup made with afang leaves, water yam, ...",Healthy,Traditional,Southern,Spicy,Affordable
4,Afang Soup,"Afang leaves, waterleaf, meat, fish",Nutritious soup made with wild spinach and wat...,Healthy,Traditional,South-South,Mild,Affordable


In [13]:
df_dishes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Food_Name         109 non-null    object
 1   Main_Ingredients  109 non-null    object
 2   Description       109 non-null    object
 3   Food_Health       109 non-null    object
 4   Food_Class        109 non-null    object
 5   Region            109 non-null    object
 6   Spice_Level       94 non-null     object
 7   Price_Range       109 non-null    object
dtypes: object(8)
memory usage: 6.9+ KB


In [14]:
df_dishes.isnull().sum()

Food_Name            0
Main_Ingredients     0
Description          0
Food_Health          0
Food_Class           0
Region               0
Spice_Level         15
Price_Range          0
dtype: int64

In [15]:
print(df_dishes.columns)

Index(['Food_Name', 'Main_Ingredients', 'Description', 'Food_Health',
       'Food_Class', 'Region', 'Spice_Level', 'Price_Range'],
      dtype='object')


In [16]:
print(f"Total number of dishes loaded: {len(df_dishes)}") 

Total number of dishes loaded: 109


In [17]:
# Combine all relevant text columns into one string per row for chunking
texts_to_chunk = [
    f"Food Name: {row['Food_Name']}. Main Ingredients: {row['Main_Ingredients']}. Description: {row['Description']}. Health Benefits: {row['Food_Health']}. Class: {row['Food_Class']}. Region: {row['Region']}" 
    for index, row in df_dishes.iterrows()
]

# Function to chunk the texts
def chunk_texts_for_rag(texts):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    text_chunks = text_splitter.create_documents(texts)
    return text_chunks

# Execute the function
dish_chunks = chunk_texts_for_rag(texts_to_chunk)

print(f"Number of chunks created for Nnobody/Nigerian-Dishes: {len(dish_chunks)}")

Number of chunks created for Nnobody/Nigerian-Dishes: 109


In [18]:
dish_chunks

[Document(metadata={}, page_content='Food Name: Abacha. Main Ingredients: African salad, utazi leaves, oil, fish. Description: Cassava-based salad with spicy fish. Health Benefits: Healthy. Class: Traditional. Region: South-East'),
 Document(metadata={}, page_content='Food Name: Abacha and Ugba (Regular). Main Ingredients: Abacha (Cassava Fufu), Ugba (Oilbean Seeds). Description: Shredded cassava fufu served with oilbean seeds.. Health Benefits: Healthy. Class: Traditional. Region: Eastern'),
 Document(metadata={}, page_content='Food Name: Abacha and Ugba. Main Ingredients: African salad, oil, fish, fermented oil bean seeds (ugba). Description: Cassava-based salad with spicy fish and fermented oil bean seeds. Health Benefits: Healthy. Class: Salad. Region: South-East'),
 Document(metadata={}, page_content='Food Name: Afang Soup (Spicy). Main Ingredients: Afang Leaves, Water yam, Seafood. Description: Spicy soup made with afang leaves, water yam, and seafood.. Health Benefits: Healthy. 

In [19]:
def download_gugging_face_embeddings():
    embeddings= HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [20]:
embeddings = download_gugging_face_embeddings()

In [21]:
query_vector = embeddings.embed_query("This is an example sentence")
print("Length", len(query_vector))

Length 384


In [34]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GROQ_API_KEY= os.environ.get("GROQ_API_KEY")
GOOGLE_CSE_ID = os.environ.get("GOOGLE_CSE_ID")
GOOGLE_CSE_API_KEY = os.environ.get("GOOGLE_CSE_API_KEY")
TWILIO_ACCOUNT_SID = os.environ.get("TWILIO_ACCOUNT_SID")
TWILIO_AUTHENTICATION_TOKEN = os.environ.get("TWILIO_AUTHENTICATION_TOKEN")
TWILIO_WHATSAPP_SID = os.environ.get("TWILIO_WHATSAPP_SID")
TWILIO_WHATSAPP_JOIN_PHRASE = os.environ.get("TWILIO_WHATSAPP_JOIN_PHRASE")
OWNER_WHATSAPP_NUMBER = os.getenv("OWNER_WHATSAPP_NUMBER")
PAYSTACK_SECRET_KEY = os.environ.get("PAYSTACK_TEST_SECRECT_KEY")
PAYSTACK_PUBLIC_SECRECT_KEY  = os.environ.get("PAYSTACK_PUBLIC_SECRECT_KEY ")
BASE_PUBLIC_URL = os.environ.get("PAYSTACK_LINK")

In [35]:
load_dotenv()

True

In [24]:
# Read API key from environment variable
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if PINECONE_API_KEY is None:
    raise ValueError("PINECONE_API_KEY environment variable is not set.")

# Initialize client
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "dashdishorderbot"
vector_dimension = 384 
metric_type = "cosine"

# Check whether the index exists
if not pc.has_index(name=index_name):
    pc.create_index(
        name=index_name,
        dimension=vector_dimension,
        metric=metric_type,
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# To get the index object later:
index = pc.Index(index_name)


## Embed each chunk and upsert the embeddings into your Pinecone index.

In [25]:
docsearch = PineconeVectorStore.from_documents(
    documents= dish_chunks,
    index_name = index_name,
    embedding= embeddings
)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {'k':3})

In [ ]:
retriever_dish = retriever.invoke("What protein with protein")

In [30]:
retriever_dish

[Document(id='26ce2727-7149-4251-8cad-c981ce1c120d', metadata={}, page_content='Food Name: Ikokore (Special). Main Ingredients: Plantains, expensive protein (prawns, crayfish), palm oil, vegetables. Description: Steamed plantains cooked in palm oil with luxurious seafood like prawns and crayfish. Health Benefits: Healthy. Class: Dish. Region: South-South'),
 Document(id='39296b13-6f95-4721-92c8-316cff8153f6', metadata={}, page_content='Food Name: Ekpang Nla. Main Ingredients: Water yam, expensive protein (assorted meats, offal), spices. Description: Special occasion dish made with water yam, a variety of expensive meats and offal, and spices. Health Benefits: Healthy. Class: Traditional. Region: South-South'),
 Document(id='778d8af2-f9f4-4ebd-8f6a-b64c7a7e7b08', metadata={}, page_content='Food Name: Miyan Taushe. Main Ingredients: Beans (black-eyed peas), tomatoes, onions, spices. Description: Spicy stewed beans with a signature red color. Health Benefits: Healthy. Class: Stew. Region:

In [33]:
# Use the API key explicitly
llm = ChatGroq(
    model="OpenAI GPT-3.5-Turbo",
    temperature=0.4,
    groq_api_key=GROQ_API_KEY
)

In [36]:
def rag_search_dishes(query: str, k: int = 3):
    """Return list of Document-like dicts from vector store."""
    results = docsearch.similarity_search(query, k=k)
    # Return standard dicts so LLM/tool chain can consume easily
    return [{"source":"rag", "content":r.page_content, "meta": getattr(r, "metadata", {})} for r in results]


In [40]:
from difflib import get_close_matches

def get_nutrition_facts(dish_name: str):
    # Step 1 — build the list of searchable names
    choices = df_foods["SearchName"].astype(str).tolist()

    # Step 2 — fuzzy match
    matches = get_close_matches(dish_name, choices, n=1, cutoff=0.3)

    if not matches:
        return None

    best_match = matches[0]

    # Step 3 — get the matched row
    row = df_foods[df_foods["SearchName"] == best_match].iloc[0]

    # Step 4 — build structured nutrition dictionary
    nutrition = {
        "match_name": row["SearchName"],
        "category": row.get("Category"),
        "local_name": row.get("LocalName"),
        "english_name": row.get("EnglishName"),
        "scientific_name": row.get("ScientificName"),

        # nutritional values (use whatever exists in your df)
        "energy_kj": row.get("ENERC_kJ"),
        "energy_kcal": row.get("ENERC_kcal") if "ENERC_kcal" in row else None,
        "protein_g": row.get("Protein_g") if "Protein_g" in row else None,
        "carbs_g": row.get("Carbs_g") if "Carbs_g" in row else None,
        "fat_g": row.get("Fat_g") if "Fat_g" in row else None,

        # vitamins (if exist)
        "vitamin_c_mg": row.get("VITC_mg"),
        "vitamin_b6_mg": row.get("VIT_B6_mg"),
        "thiamin_mg": row.get("THIA_mg"),
        "riboflavin_mg": row.get("RIBF_mg"),

        # reference ID
        "ref_id": row.get("REFID")
    }

    return nutrition


In [41]:
info = get_nutrition_facts("jollof rice")
print(info)

{'match_name': 'Watermelon juice', 'category': 'Condiments and Spices', 'local_name': None, 'english_name': 'Watermelon juice', 'scientific_name': None, 'energy_kj': np.float64(125.19), 'energy_kcal': np.float64(29.56), 'protein_g': None, 'carbs_g': None, 'fat_g': None, 'vitamin_c_mg': np.float64(16.76), 'vitamin_b6_mg': np.float64(nan), 'thiamin_mg': np.float64(0.05), 'riboflavin_mg': np.float64(0.04), 'ref_id': '15'}
